In [ ]:
!pip install -U fsspec==2023.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.6.0 which is incompatible.


In [ ]:
!pip install transformers datasets evaluate accelerate -q

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    GPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer
)
import math
import torch

In [ ]:
# Load WikiText-2
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Load tokenizer and set pad token
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def filter_empty(example):
    return len(example["text"].strip()) > 0  # Remove blank or whitespace-only text

dataset = dataset.filter(filter_empty)

Filter:   0%|          | 0/4358 [00:00<?, ? examples/s]

Filter:   0%|          | 0/36718 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/2891 [00:00<?, ? examples/s]

Map:   0%|          | 0/23767 [00:00<?, ? examples/s]

Map:   0%|          | 0/2461 [00:00<?, ? examples/s]

In [ ]:
# Load GPT-2 with language modeling head
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

# Collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=50,
    save_total_limit=2,
    push_to_hub=False,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].select(range(2000)),  # subset for Colab
    eval_dataset=tokenized_datasets["validation"].select(range(10)),
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/tmp/ipython-input-8-954226773.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: arvindbhimanpalli12 (arvindbhimanpalli12-iit-bombay) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.285600,3.341316
2,2.661600,3.368182


TrainOutput(global_step=2000, training_loss=3.090002914428711, metrics={'train_runtime': 549.4458, 'train_samples_per_second': 7.28, 'train_steps_per_second': 3.64, 'total_flos': 312778810368000.0, 'train_loss': 3.090002914428711, 'epoch': 2.0})

In [ ]:
eval_results = trainer.evaluate()
perplexity = math.exp(eval_results["eval_loss"])
print(f"\nPerplexity: {perplexity:.2f}")


Perplexity: 29.03


In [ ]:
from transformers import pipeline

text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
output = text_generator("The future of AI is", max_length=30, do_sample=True)
print(output[0]["generated_text"])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


The future of AI is uncertain. Many scientists believe it will revolutionise the way we conduct intelligence, with an emphasis on human intelligence. But some scientists think that the AI revolution will cause us to lose the ability to control what is happening on the battlefield. 

Human intelligence is highly complex. It has many tools and processes that are not understood by humans. 

If human intelligence is to be developed from scratch, it must be able to recognise the human brain as it travels across the universe, learning from it so it can learn to play with its environment. 

 

A team of scientists has developed a new form of AI called the Singularity Project, a project of the Advanced Research Projects Agency ( ARPA ) of the Department of Defense ( Darpa ) to develop new and more powerful tools for studying human behaviour, and for identifying artificial intelligence. 

They have also developed a small but powerful " self @-@ determining " system that can predict the future w

In [ ]:
import numpy as np
from tqdm import tqdm

def compute_top_k_accuracy(model, dataset, k=5, max_samples=200):
    model.eval()
    correct = 0
    total = 0

    for example in tqdm(dataset.select(range(min(max_samples, len(dataset))))):
        input_ids = torch.tensor(example["input_ids"]).unsqueeze(0).to(model.device)
        attention_mask = torch.tensor(example["attention_mask"]).unsqueeze(0).to(model.device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits  # shape: (1, seq_len, vocab_size)

        # We evaluate the prediction of the **last token**
        last_token_logits = logits[0, -2]  # logits before the final input token
        true_token = input_ids[0, -1]      # the actual next token in the sequence

        top_k_preds = torch.topk(last_token_logits, k).indices
        if true_token in top_k_preds:
            correct += 1
        total += 1

    accuracy = correct / total if total > 0 else 0
    print(f"Top-{k} Accuracy: {accuracy * 100:.2f}%")
    return accuracy


In [ ]:
compute_top_k_accuracy(model, tokenized_datasets["validation"], k=5)

100%|██████████| 200/200 [00:02<00:00, 72.29it/s]

Top-5 Accuracy: 100.00%


1.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model.save_pretrained("/content/drive/MyDrive/Next-Word-Prediction")
tokenizer.save_pretrained("/content/drive/MyDrive/Next-Word-Prediction")

Mounted at /content/drive


('/content/drive/MyDrive/Next-Word-Prediction/tokenizer_config.json',
 '/content/drive/MyDrive/Next-Word-Prediction/special_tokens_map.json',
 '/content/drive/MyDrive/Next-Word-Prediction/vocab.json',
 '/content/drive/MyDrive/Next-Word-Prediction/merges.txt',
 '/content/drive/MyDrive/Next-Word-Prediction/added_tokens.json',
 '/content/drive/MyDrive/Next-Word-Prediction/tokenizer.json')